In [1]:
import os
import sqlite3

In [2]:
conn = sqlite3.connect('stcv.sqlite')

In [3]:
c = conn.cursor()

In [4]:
query = """
SELECT DISTINCT
    title.cloi as identifier,
    COUNT(title.cloi) as id_count,
    author_vw as author_standardized,
    author_zvwr as author_original,
    author_zbd as author_dates,
    corporateauthor_nm as corporateauthor_standardized,
    corporateauthor_zvwr as corporateauthor_original,
    title_ti as title_title
    FROM title
LEFT JOIN author on author.cloi = title.cloi
LEFT JOIN collation on collation.cloi = title.cloi
LEFT JOIN corporateauthor on corporateauthor.cloi = title.cloi
LEFT JOIN edition on edition.cloi = title.cloi
LEFT JOIN impressum on impressum.cloi = title.cloi
LEFT JOIN language on language.cloi = title.cloi
LEFT JOIN number on number.cloi = title.cloi
GROUP BY identifier
HAVING ID_COUNT=2
LIMIT 100
"""

In [5]:
c.execute(query)

In [6]:
data = [row for row in c.fetchall()]
for row in data:
    print(row)

('c:stcv:12854444', 2, 'Ernst, Simon-Pierre', 'S.P. Ernst', '1744-1817', None, None, 'Apologie des ministres du culte, qui ont prêté la déclaration exigée par la loi du 7 vendém. an 4. contre les critiques de mm. Dédoyar & Vanhoren, les Motifs de Malines & autres brochures')
('c:stcv:12854501', 2, 'Ernst, Simon-Pierre', 'S.P. Ernst', '1744-1817', None, None, 'Observations sur la déclaration exigée des ministres des cultes, en vertu de la loi du 7 vendémiaire, an 4')
('c:stcv:12854549', 2, 'Gisbert, Blaise', 'Gisbert', '1657-1731', None, None, "l'Eloquence chrétienne dans l'idée et dans la pratique")
('c:stcv:12854587', 2, None, None, None, None, None, 'Verscheyde leeringen en exempelen der oude vaders')
('c:stcv:12854599', 2, None, None, None, None, None, 'Godtvruchtige leeringen en gebeden voor de eerste communie')
('c:stcv:12857052', 2, 'Courvoisier, Jean-Jacques', 'Iean Iacques Couruoisier', '', None, None, "Le prince immortel tiré sur la vie & la fin glorieuse de son altesse royale

In [7]:
data

[('c:stcv:12854444',
  2,
  'Ernst, Simon-Pierre',
  'S.P. Ernst',
  '1744-1817',
  None,
  None,
  'Apologie des ministres du culte, qui ont prêté la déclaration exigée par la loi du 7 vendém. an 4. contre les critiques de mm. Dédoyar & Vanhoren, les Motifs de Malines & autres brochures'),
 ('c:stcv:12854501',
  2,
  'Ernst, Simon-Pierre',
  'S.P. Ernst',
  '1744-1817',
  None,
  None,
  'Observations sur la déclaration exigée des ministres des cultes, en vertu de la loi du 7 vendémiaire, an 4'),
 ('c:stcv:12854549',
  2,
  'Gisbert, Blaise',
  'Gisbert',
  '1657-1731',
  None,
  None,
  "l'Eloquence chrétienne dans l'idée et dans la pratique"),
 ('c:stcv:12854587',
  2,
  None,
  None,
  None,
  None,
  None,
  'Verscheyde leeringen en exempelen der oude vaders'),
 ('c:stcv:12854599',
  2,
  None,
  None,
  None,
  None,
  None,
  'Godtvruchtige leeringen en gebeden voor de eerste communie'),
 ('c:stcv:12857052',
  2,
  'Courvoisier, Jean-Jacques',
  'Iean Iacques Couruoisier',
  '',

In [8]:
stcv_dict = {}
for row in data:
    key, value = row[0], row[-1]
    stcv_dict[key] = value

In [9]:
stcv_dict

{'c:stcv:12854444': 'Apologie des ministres du culte, qui ont prêté la déclaration exigée par la loi du 7 vendém. an 4. contre les critiques de mm. Dédoyar & Vanhoren, les Motifs de Malines & autres brochures',
 'c:stcv:12854501': 'Observations sur la déclaration exigée des ministres des cultes, en vertu de la loi du 7 vendémiaire, an 4',
 'c:stcv:12854549': "l'Eloquence chrétienne dans l'idée et dans la pratique",
 'c:stcv:12854587': 'Verscheyde leeringen en exempelen der oude vaders',
 'c:stcv:12854599': 'Godtvruchtige leeringen en gebeden voor de eerste communie',
 'c:stcv:12857052': "Le prince immortel tiré sur la vie & la fin glorieuse de son altesse royale don Ferdinand d'Avstriche",
 'c:stcv:12857069': 'Trivmphvs rosarii a sede apostolica decretvs sodalitati B. virginis Mariæ',
 'c:stcv:12857131': 'In statvas antiqvitatis et elegantias Arvndellianas cento Maronianvs',
 'c:stcv:12857158': 'Inde[x] librorvm qvi in typographia Plantiniana excvsi venales nvnc exstant',
 'c:stcv:1285

In [10]:
conn.close()

In [11]:
import urllib.parse
import urllib.request
import urllib.error
from urllib.parse import quote

import json
import sys
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import lxml
import lxml.etree
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import fromstring, ElementTree


In [12]:
# from xml.etree import ElementTree

In [13]:
HBP = "https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query="

In [14]:
def clean(string: str) -> str:
    """
    clean input string and URL encode
    """
    string = string.strip()
    string = string.casefold()
    string = urllib.parse.quote(string)
    return string


def query_hbp(dict):
    hbp_lists = []
    for value in stcv_dict.values():
        temp_list = []
        url = HBP + clean(value) + '&startRecord=1&maximumRecords=10&recordSchema=marcxml'
        try:
            with urllib.request.urlopen(url) as query:
                f = query.read()
                tree = lxml.etree.fromstring(f, parser=None)
                root = tree.getroottree()
                for element in tree.iter('{http://www.loc.gov/MARC21/slim}datafield'):
                    for item in element.items():
                        if item == ('tag', '035'):
                            for child in element:
                                for item in child.items():
                                    if item == ('code', 'a'):
                                        temp_list.append(child.text)
                hbp_lists.append(temp_list)
        except urllib.error.HTTPError as HTTPerr:
            exit(HTTPerr.code)
            print('Something went wrong')
        except urllib.error.URLError as URLerr:
            exit(URLerr)
            print('Something went wrong')
    return(hbp_lists)
    print(len(hbp_lists))
    
query_hbp(stcv_dict)
    

[[],
 ['BE-AnVE.c:stcv:12913022',
  'BE-AnVE.c:stcv:12854501',
  'DE-604.VK.BV021245268',
  'BE-KBR00.CAT.1_1397836',
  'FR-751131015.CG.FRBNF363412130000000',
  'FR-751131015.CG.FRBNF303526230000006'],
 ['(OCoLC)253314165',
  'DE-601.GVK.454683766',
  '(OCoLC)168782481 ',
  'BE-AnVE.c:stcv:12854549',
  'BE-KBR00.CAT.1_1732080',
  'BE-KBR00.CAT.1_1604695',
  'BE-KBR00.CAT.1_1506014',
  'DE-603.351439676',
  'FR-341725201.SUDOC.099981661',
  '(OCoLC)244521759 ',
  'SpMaUC.BH.b22061861',
  'FR-341725201.SUDOC.135439248',
  '(OCoLC)257480982',
  'DE-601.GVK.153387947',
  '(OCoLC)169122583 '],
 ['BE-AnVE.c:stcv:12854587', 'NL-0100030000.STCN.328229482'],
 ['BE-AnVE.c:stcv:12854599',
  'BE-AnVE.c:stcv:3207115',
  'NL-0100030000.STCN.328577863',
  'BE-AnVE.c:stcv:6170080',
  'BE-AnVE.c:stcv:3207179',
  'BE-AnVE.c:stcv:3139737',
  'NL-0100030000.STCN.317873369',
  'NL-0100030000.STCN.316078743',
  'NL-0100030000.STCN.314809163'],
 ['BE-AnVE.c:stcv:12857052'],
 [],
 ['BE-AnVE.c:stcv:12857131']

In [15]:
hbp_lists = query_hbp(stcv_dict)
len(hbp_lists)

100

In [16]:
def zip_stcv_hbp(list1, list2):
    list1 = list(stcv_dict.keys())
    list2 = hbp_lists
    matches = dict(zip(list1, list2))
    return matches

In [17]:
stcv_list = list(stcv_dict.keys())

In [18]:
matching_ids = zip_stcv_hbp(stcv_list, hbp_lists)

In [19]:
matching_ids

{'c:stcv:12854444': [],
 'c:stcv:12854501': ['BE-AnVE.c:stcv:12913022',
  'BE-AnVE.c:stcv:12854501',
  'DE-604.VK.BV021245268',
  'BE-KBR00.CAT.1_1397836',
  'FR-751131015.CG.FRBNF363412130000000',
  'FR-751131015.CG.FRBNF303526230000006'],
 'c:stcv:12854549': ['(OCoLC)253314165',
  'DE-601.GVK.454683766',
  '(OCoLC)168782481 ',
  'BE-AnVE.c:stcv:12854549',
  'BE-KBR00.CAT.1_1732080',
  'BE-KBR00.CAT.1_1604695',
  'BE-KBR00.CAT.1_1506014',
  'DE-603.351439676',
  'FR-341725201.SUDOC.099981661',
  '(OCoLC)244521759 ',
  'SpMaUC.BH.b22061861',
  'FR-341725201.SUDOC.135439248',
  '(OCoLC)257480982',
  'DE-601.GVK.153387947',
  '(OCoLC)169122583 '],
 'c:stcv:12854587': ['BE-AnVE.c:stcv:12854587',
  'NL-0100030000.STCN.328229482'],
 'c:stcv:12854599': ['BE-AnVE.c:stcv:12854599',
  'BE-AnVE.c:stcv:3207115',
  'NL-0100030000.STCN.328577863',
  'BE-AnVE.c:stcv:6170080',
  'BE-AnVE.c:stcv:3207179',
  'BE-AnVE.c:stcv:3139737',
  'NL-0100030000.STCN.317873369',
  'NL-0100030000.STCN.316078743',
 